In [1]:
!pip install pandas dash

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Import necessary libraries
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

C:\Users\ASUS PC\AppData\Local\Temp\ipykernel_16116\761955065.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\ASUS PC\AppData\Local\Temp\ipykernel_16116\761955065.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [3]:
# Load the dataset
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df = pd.read_csv(url)

# Initialize Dash app
app = dash.Dash(__name__)

In [4]:
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [5]:
# Task 1: Dropdown for launch sites

# Task 1: Dropdown for launch sites with 'All Sites' option
launch_sites = [{'label': 'All Sites', 'value': 'ALL'}]
launch_sites.extend([{'label': site, 'value': site} for site in spacex_df['Launch Site'].unique()])

app.layout = html.Div([
    dcc.Dropdown(
        id='launch-site-dropdown',
        options=launch_sites,
        value='ALL'  # Default value
    ),
    dcc.Graph(id='success-pie-chart'),
    dcc.RangeSlider(
        id='payload-range-slider',
        min=0,
        max=10000,
        step=1000,
        marks={i: str(i) for i in range(0, 10001, 1000)},
        value=[min(spacex_df['Payload Mass (kg)']), max(spacex_df['Payload Mass (kg)'])]
    ),
    dcc.Graph(id='success-payload-scatter-chart'),
])

# Task 2: Callback for the pie chart
@app.callback(
    Output('success-pie-chart', 'figure'),
    [Input('launch-site-dropdown', 'value')]
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        # Group by 'Launch Site' and sum the 'class' column
        success_sum = spacex_df.groupby('Launch Site')['class'].sum().reset_index()
        success_sum.columns = ['Launch Site', 'Total Successful Launches']

        # Create the pie chart for all sites
        fig = px.pie(success_sum, names='Launch Site', values='Total Successful Launches',
                     title='Total Successful Launches by Site')
    else:
        # Filter for the selected launch site
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]

        # Group by 'class' and count the occurrences
        success_counts = filtered_df['class'].value_counts().reset_index()
        success_counts.columns = ['class', 'count']

        # Create the pie chart for a specific site
        fig = px.pie(success_counts, names='class', values='count',
                     title=f'Launch Success Rates for {selected_site}')
    return fig

# Task 4: Callback for the scatter chart
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('payload-range-slider', 'value')]
)
def update_scatter_chart(payload_range):
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload_range[0]) & 
                            (spacex_df['Payload Mass (kg)'] <= payload_range[1])]
    fig = px.scatter(filtered_df, 
                     x='Payload Mass (kg)', 
                     y='class', 
                     color='Booster Version',
                     title='Success by Payload and Booster Version')
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)